In [1]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
#reading data from csv file
import pandas as pd
df = pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
#selecting first 5000 reviews only
df2 = df.iloc[0:5000,:]

In [4]:
#importing required libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [5]:
#initializing word net lemmatizer
lemmatizer = WordNetLemmatizer()

In [6]:
#data preprocessing
corpus = []
for i in range(len(df2['review'])):
    review = re.sub('[^a-zA-Z]',' ', df2['review'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words("english")]
    review = ' '.join(review)
    corpus.append(review)

In [7]:
#checking length of corpus
len(corpus)

5000

In [8]:
#importing and initializing count vectorizer for bag of words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(max_features = 2500, ngram_range = (1,2), binary = True)

In [9]:
#separating input and output data
X = corpus
y = df2['sentiment'].map({'positive':1, 'negative':0})

In [10]:
#splitting data into train and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [ ]:
X_train

In [11]:
#transforming X_train and X_test into bag of words
X_train_c = cv.fit_transform(X_train)
X_test_c = cv.transform(X_test)

In [13]:
X_train_c.shape
X_test_c.shape

(1000, 2500)

In [15]:
#importing initializing and fitting MultinomialNB to training data
from sklearn.naive_bayes import MultinomialNB
mod = MultinomialNB().fit(X_train_c.toarray(), y_train)

In [16]:
#making predictions and checking accuracy
from sklearn.metrics import accuracy_score, classification_report
y_preds = mod.predict(X_test_c)
accuracy = round(accuracy_score(y_test, y_preds),2)
accuracy

0.86

In [17]:
#printing classification report
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.87      0.86      0.86       502
           1       0.86      0.87      0.86       498

    accuracy                           0.86      1000
   macro avg       0.86      0.86      0.86      1000
weighted avg       0.86      0.86      0.86      1000



In [18]:
#implementing tfidf vectorizer
tf = TfidfVectorizer(max_features = 2500, ngram_range = (1,2), binary = True)
X_train_t = tf.fit_transform(X_train)
X_test_t = tf.transform(X_test)

In [19]:
#fitting MultinomialNB on training data
mod2 = MultinomialNB().fit(X_train_t, y_train)

In [20]:
#making predictions on test data
y_preds2 = mod2.predict(X_test_t)

In [21]:
#checking accuracy 
accuracy = round(accuracy_score(y_test, y_preds2),2)
accuracy

0.86

In [22]:
#classification report
print(classification_report(y_test, y_preds2))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       502
           1       0.86      0.86      0.86       498

    accuracy                           0.86      1000
   macro avg       0.86      0.86      0.86      1000
weighted avg       0.86      0.86      0.86      1000



### Word2Vec

In [23]:
#importing required libraries
import gensim 
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [24]:
#creating list of words
words = []
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for w in sent_token:
        words.append(simple_preprocess(w))
    

In [25]:
#fitting word2vec on list of words
from gensim.models import Word2Vec
model = Word2Vec(words, window = 10, min_count = 2)

In [26]:
#all words in out model
model.wv.index_to_key

['br',
 'movie',
 'film',
 'one',
 'like',
 'time',
 'good',
 'character',
 'get',
 'story',
 'would',
 'even',
 'make',
 'see',
 'really',
 'well',
 'scene',
 'much',
 'bad',
 'first',
 'people',
 'also',
 'way',
 'great',
 'show',
 'made',
 'thing',
 'life',
 'go',
 'could',
 'think',
 'watch',
 'know',
 'plot',
 'never',
 'actor',
 'look',
 'little',
 'seen',
 'love',
 'many',
 'two',
 'year',
 'say',
 'acting',
 'best',
 'end',
 'ever',
 'come',
 'take',
 'man',
 'work',
 'better',
 'still',
 'something',
 'want',
 'part',
 'lot',
 'real',
 'director',
 'back',
 'watching',
 'performance',
 'give',
 'find',
 'woman',
 'though',
 'going',
 'old',
 'new',
 'guy',
 'play',
 'funny',
 'nothing',
 'girl',
 'day',
 'actually',
 'role',
 'another',
 'feel',
 'horror',
 'every',
 'point',
 'minute',
 'pretty',
 'star',
 'world',
 'around',
 'quite',
 'comedy',
 'fact',
 'thought',
 'young',
 'enough',
 'must',
 'big',
 'cast',
 'got',
 'however',
 'long',
 'line',
 'family',
 'action',
 's

In [27]:
#count of all words
model.corpus_count

5000

In [28]:
#most similar words
model.wv.similar_by_word('scene')

[('sequence', 0.8441349267959595),
 ('shot', 0.8297575116157532),
 ('action', 0.8251043558120728),
 ('camera', 0.801484227180481),
 ('fight', 0.7788670659065247),
 ('rather', 0.7747578024864197),
 ('moment', 0.7726181149482727),
 ('twist', 0.7668361067771912),
 ('unintended', 0.7652679085731506),
 ('cadaver', 0.7627105116844177)]

In [30]:
#function for average word2vec
import numpy as np
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis = 0)

In [31]:
#applying average word2 vec on data
from tqdm import tqdm
X = []
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:33<00:00, 147.38it/s]


In [32]:
#creating numpy array of input data
X_new = np.array(X)

In [33]:
#checking shape
X_new.shape

(5000, 100)

In [37]:
#splitting data into train and test data
X_trainw, X_testw, y_trainw, y_testw = train_test_split(X_new, y, test_size = 0.2, random_state = 3)

In [47]:
#fitting random forest classifier on training data
from sklearn.ensemble import RandomForestClassifier
mod3 = RandomForestClassifier().fit(X_trainw, y_trainw)

In [48]:
#making predictions on test data
preds = mod3.predict(X_testw)

In [49]:
#accuracy on test data
round(accuracy_score(y_testw, preds),2)

0.76

In [54]:
#classification report
print(classification_report(y_testw, preds))

              precision    recall  f1-score   support

           0       0.75      0.77      0.76       477
           1       0.78      0.76      0.77       523

    accuracy                           0.76      1000
   macro avg       0.76      0.76      0.76      1000
weighted avg       0.76      0.76      0.76      1000



In [50]:
#importing and initializing knn classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [51]:
#fitting knn model on training data
knn.fit(X_trainw, y_trainw)

KNeighborsClassifier()

In [52]:
#making predictions on test data
knn_preds = knn.predict(X_testw)

In [55]:
#accuracy
round(accuracy_score(y_testw, knn_preds),2)

0.7

In [56]:
#classification report
print(classification_report(y_testw, knn_preds))

              precision    recall  f1-score   support

           0       0.67      0.72      0.70       477
           1       0.73      0.68      0.70       523

    accuracy                           0.70      1000
   macro avg       0.70      0.70      0.70      1000
weighted avg       0.70      0.70      0.70      1000

